In [ ]:
source("Main.R")
source("Conf.R")
source("Utilities.R")

geneCutOff=4
targetCutOff=15
nFactors=15

In [ ]:
myR2 <- function(preds, actual){
    rss <- sum((preds - actual) ^ 2)  ## residual sum of squares
    tss <- sum((actual - mean(actual)) ^ 2)  ## total sum of squares
    rsq <- 1 - rss/tss
}


In [ ]:
selCoefs <- read.csv("./../TextFiles/SignificantCoefMatrix.csv", 
                     row.names = 1)
selCoefs <- t(selCoefs)
KOGenes <-  colnames(selCoefs)
KOGenes <- sapply(KOGenes, function(x){strsplit(x,"_")[[1]][2]})
colnames(selCoefs) <- KOGenes

In [ ]:
FOBIasymp(selCoefs,k = 20, type = "S1", model = "ICA")

In [ ]:
FOBIboot(X=selCoefs, k=20, n.boot = 200, s.boot = "B1")

In [ ]:
test <- FOBIladle(selCoefs)
test
summary(test)
plot(test)

In [ ]:
ladleplot(test)

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)

pdf("./../SupplementaryFigures/S_5C.pdf", width=6, height=6)
ggladleplot(test, crit="gn")+theme_minimal()+
scale_x_continuous(breaks=seq(0, 40, 5))+
xlim(0,40)+xlab("Number of components")+
theme(axis.text=element_text(size=15),axis.title=element_text(size=20))+geom_vline(xintercept=15, color="red")
dev.off()

In [ ]:
randomIndexes = sample(1:342489, 20000, replace=TRUE)

In [ ]:
selCoefsTmp = copy(selCoefs)

In [ ]:
selCoefsTmp[randomIndexes] = NA

In [ ]:
k <- icaimax(selCoefs, 
             nc = 15, 
             center = T)

In [ ]:
factorCov = cor((k$S), method = "pearson")

In [ ]:
diag(factorCov) <- 0

In [ ]:
options(repr.plot.width=9, repr.plot.height=8)

col3 = colorRampPalette(c("navy", "white", "red"))(50)
corrplot::corrplot(factorCov, order = 'AOE', col=col3, 
                   method = 'ellipse', is.corr = FALSE, 
                   tl.cex = 1.5, diag = FALSE, 
                   type = 'upper')

In [ ]:
actualVals = melt(selCoefs)$value
allR2 <- list()

for(i in 2:100){
    k <- icaimax(selCoefs, nc = i, center = T)
    redVals = k$S %*% t(k$M)
    reconstVal = melt(redVals)$value
    R2 = myR2(reconstVal, actualVals)
    allR2 <- lappend(allR2, c("k"=i, "R2"=R2))
}

In [ ]:
allR2 <- data.frame(do.call(rbind,allR2))

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)

pdf("./../SupplementaryFigures/S_5D.pdf", width=6, height=6)

ggplot(data=allR2, aes(x=k, y=R2)) +
  geom_line()+
  geom_point()+geom_vline(xintercept=15, color="red")+theme_minimal()+
xlab("Number of components")+ ylab("Explained variance in beta matrix")+
scale_x_continuous(breaks=seq(0, 100, 15))+
scale_y_continuous(breaks=seq(0, 0.8, 0.1))+
theme(axis.text=element_text(size=15),axis.title=element_text(size=20))

dev.off()